Import relevant Python libraries for the data extraction.

In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup
import urllib.request

#!conda install -c conda-forge lxml --yes

import lxml

print('Libraries imported.')

Libraries imported.


Store the Wiki URL in url and call urllib package to read the url

In [50]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urllib.request.urlopen(url)

Read the page using BeautifulSoup package

In [51]:
soup = BeautifulSoup(page, "html5lib")

Extract all tables from soup

In [52]:
all_tables=soup.find_all("table")

Store relevant table as right_table

In [53]:
right_table=soup.find('table', class_='wikitable sortable')

Extract each column and append it as A,B and C

In [54]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Merge A, B and C as a single dataframe under columns Postal Code, Borough, Neighborhood. We remove all 'Not assigned' boroughs and assign 'Not assigned' neighborhoods to their borough names.

In [55]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df['Postal Code'] = df['Postal Code'].astype(str).str[:-1]
df['Borough'] = df['Borough'].astype(str).str[:-1]
df['Neighborhood'] = df['Neighborhood'].astype(str).str[:-1]

df_cleaned_br = df[df['Borough'] != 'Not assigned']

for index,row in df_cleaned_br.iterrows():
   if(row['Neighborhood'] == 'Not assigned'):
        row['Neighborhood'] = row['Borough']

df_cleaned_br

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Check the shape of the final dataframe

In [56]:
df_cleaned_br.shape

(103, 3)

Install geocoder package

In [58]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         837 KB

The following NEW packages will be INSTALLED:

  click              conda-forge/noarch::click-7.1.2-pyh9f0ad1d_0
  decorator          conda-forge/noarch::decorator-4.4.2-py_0
  future             conda-forge/linux-64::future-0.18.2-py36h9f0ad1d_1
  geocoder       

Download postal code csv and store it in dataframe

In [69]:
postal_code_ll = pd.read_csv('http://cocl.us/Geospatial_data')
postal_code_ll['Postal Code'] = postal_code_ll['Postal Code'].astype(str)
postal_code_ll['Latitude'] = postal_code_ll['Latitude'].astype(str)
postal_code_ll['Longitude'] = postal_code_ll['Longitude'].astype(str)
postal_code_ll

,Postal Code,Latitude,Longitude
0,M1B,43.806686299999996,-79.19435340000001
1,M1C,43.7845351,-79.16049709999999
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.21691740000001
4,M1H,43.773136,-79.23947609999999
5,M1J,43.7447342,-79.23947609999999
6,M1K,43.7279292,-79.26202940000002
7,M1L,43.711111700000004,-79.2845772
8,M1M,43.716316,-79.23947609999999
9,M1N,43.692657000000004,-79.2648481


Join existing neighborhood table with table containing lat long, to get latitude and longitude for each neighborhood

In [74]:
result = pd.merge(df_cleaned_br, postal_code_ll,
                  left_on= 'Postal Code',
                  right_on = 'Postal Code',
                  how = 'left'
)
result

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.725882299999995,-79.31557159999998
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718517999999996,-79.46476329999999
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623015,-79.3894938
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6678556,-79.53224240000002
6,M1B,Scarborough,"Malvern, Rouge",43.806686299999996,-79.19435340000001
7,M3B,North York,Don Mills,43.745905799999996,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063972,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6571618,-79.37893709999999


Get shape of resultant dataframe

In [75]:
result.shape

(103, 5)